SIDE NOTE: would love to get any critiques, hints, tips, oberservations on any level not just on ML (but obviously ML as the priority).  Beginner Python learner (though I took the Full Stack Course) so any suggestions there are welcome.  Thanks.

Will need to import these libs plus setting some global variables

In [1]:
import sys
import os
import numpy as np
import renders as rs
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from IPython.display import display # Allows the use of display() for DataFrames
from sklearn.externals import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
import matplotlib.cm as cm
# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

# Might need to change the path of the included libraries.
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/match_stats.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_model.py')
sys.path.append('/anaconda/envs/stats/lib/python3.5/site-packages')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/model_libs.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_data.py')
sys.path.append('/Users/senzari/Machine_Learning/stats')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats')
# print(sys.path)
os.chdir('/Users/senzari/Machine_Learning/stats/src')
#print(os.getcwd())

from stats import form_data, match_stats, model_libs, form_model, predict_matches

# Variables
round_number = 27 # for MLS only
target_col = 'points'
ignore_cols = ['games_played', 'match_id', 'team_id', 'team_name', 'opp_id', 'opp_name', 'scheduled']
all_models = ['log', 'svc', 'gmm', 'knn', 'gnb']

""" this variable 'testing' should be false if using CSV's and not pulling from the database. """
testing = True

INITIALIZED...


Pulling in the data either from the Database or the CSV (CSV for testers).  Data has some added features in between the database and 'raw_data'.  Essentially trying 3 different versions of the data in this set.  One version contains all the features.  Another version has the standard features plus the calculated features of the home/away stats for a matchup.  The last version has the standard features plus the 'extended features' which not only includes the stats of the two teams in the match up but also the current teams previous opponents and the current teams opponents of the team opponents in that matchup.

In [2]:
data_csv = 'round' + str(round_number) + '.csv'

if testing:
    raw_data = form_data.run_data(round_number)
    raw_data.to_csv(data_csv)
    print("Raw Data Saved to CSV")
else:
    #Reading in a CSV adds the first index column
    raw_data = pd.read_csv(data_csv)
    raw_data = raw_data.drop(raw_data.columns[[0]], axis=1)

pd.set_option("display.max_columns", 85)

print('Data Loaded...')

ROUND 2 :: TEAM ID 21
ROUND 3 :: TEAM ID 21
ROUND 4 :: TEAM ID 21
ROUND 5 :: TEAM ID 21
ROUND 6 :: TEAM ID 21
ROUND 7 :: TEAM ID 21
ROUND 8 :: TEAM ID 21
ROUND 9 :: TEAM ID 21
ROUND 10 :: TEAM ID 21
ROUND 11 :: TEAM ID 21
ROUND 12 :: TEAM ID 21
ROUND 13 :: TEAM ID 21
ROUND 14 :: TEAM ID 21
ROUND 15 :: TEAM ID 21
ROUND 16 :: TEAM ID 21
ROUND 18 :: TEAM ID 21
ROUND 19 :: TEAM ID 21
ROUND 21 :: TEAM ID 21
ROUND 23 :: TEAM ID 21
ROUND 24 :: TEAM ID 21
ROUND 25 :: TEAM ID 21
ROUND 26 :: TEAM ID 21
ROUND 2 :: TEAM ID 22
ROUND 3 :: TEAM ID 22
ROUND 5 :: TEAM ID 22
ROUND 6 :: TEAM ID 22
ROUND 7 :: TEAM ID 22
ROUND 8 :: TEAM ID 22
ROUND 9 :: TEAM ID 22
ROUND 10 :: TEAM ID 22
ROUND 11 :: TEAM ID 22
ROUND 12 :: TEAM ID 22
ROUND 13 :: TEAM ID 22
ROUND 15 :: TEAM ID 22
ROUND 16 :: TEAM ID 22
ROUND 18 :: TEAM ID 22
ROUND 19 :: TEAM ID 22
ROUND 21 :: TEAM ID 22
ROUND 22 :: TEAM ID 22
ROUND 23 :: TEAM ID 22
ROUND 24 :: TEAM ID 22
ROUND 25 :: TEAM ID 22
ROUND 26 :: TEAM ID 22
ROUND 2 :: TEAM ID 23
ROUN

/Users/senzari/.local/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


ROUND 8 :: TEAM ID 23
ROUND 9 :: TEAM ID 23
ROUND 10 :: TEAM ID 23
ROUND 11 :: TEAM ID 23
ROUND 12 :: TEAM ID 23
ROUND 13 :: TEAM ID 23
ROUND 14 :: TEAM ID 23
ROUND 15 :: TEAM ID 23
ROUND 16 :: TEAM ID 23
ROUND 17 :: TEAM ID 23
ROUND 18 :: TEAM ID 23
ROUND 19 :: TEAM ID 23
ROUND 21 :: TEAM ID 23
ROUND 22 :: TEAM ID 23
ROUND 23 :: TEAM ID 23
ROUND 24 :: TEAM ID 23
ROUND 25 :: TEAM ID 23
ROUND 26 :: TEAM ID 23
ROUND 2 :: TEAM ID 24
ROUND 3 :: TEAM ID 24
ROUND 4 :: TEAM ID 24
ROUND 6 :: TEAM ID 24
ROUND 7 :: TEAM ID 24
ROUND 8 :: TEAM ID 24
ROUND 9 :: TEAM ID 24
ROUND 10 :: TEAM ID 24
ROUND 11 :: TEAM ID 24
ROUND 12 :: TEAM ID 24
ROUND 13 :: TEAM ID 24
ROUND 14 :: TEAM ID 24
ROUND 15 :: TEAM ID 24
ROUND 16 :: TEAM ID 24
ROUND 17 :: TEAM ID 24
ROUND 18 :: TEAM ID 24
ROUND 19 :: TEAM ID 24
ROUND 21 :: TEAM ID 24
ROUND 22 :: TEAM ID 24
ROUND 23 :: TEAM ID 24
ROUND 24 :: TEAM ID 24
ROUND 25 :: TEAM ID 24
ROUND 26 :: TEAM ID 24
ROUND 2 :: TEAM ID 25
ROUND 3 :: TEAM ID 25
ROUND 5 :: TEAM ID 25


## Training

Functions that trains basic models (and saves them if need be).  'load_models' will  load exisiting models to save time in testing.

In [3]:
# Removes Columns to Ignore and Splits the Target Column
def split_target(data):
    td = model_libs._clone_and_drop(data, ignore_cols)
    (y, X) = model_libs._extract_target(td, target_col)
    return X, y

"""No Cross-Validation and No Tuning"""
def train_models(round_num, X, y, models):
    
    if os.path.isdir("/models/" + str(round_num)):
        print('Making New Directory for the Round')
        os.chdir('/Users/senzari/Machine_Learning/stats/src/models')
        os.makedirs(str(round_num))
        os.chdir('/Users/senzari/Machine_Learning/stats/src')
        
    finished_models = []
    
    for i in models:
        
        model_round = 'models/' + str(round_num) + '/' + str(i) + '_round_' + str(round_num) + '.pk1'
        
        
        if i == 'log':
            log = form_model.build_model(X, y, i)
            joblib.dump(log, model_round)
            finished_models.append(log)
        elif i == 'svc':
            svc = form_model.build_model(X, y, i)
            joblib.dump(svc, model_round)
            finished_models.append(svc)
        elif i == 'gmm':
            gmm = form_model.build_model(X, y, i)
            joblib.dump(gmm, model_round)
            finished_models.append(gmm)
        elif i == 'knn':
            kmeans = form_model.build_model(X, y, i)
            joblib.dump(kmeans, model_round)
            finished_models.append(kmeans)
        elif i == 'gnb':
            gnb = form_model.build_model(X, y, i)
            joblib.dump(gnb, model_round)
            finished_models.append(gnb)
    
    return finished_models
        
        
def load_models(round_num):
    
    loaded_models = []
    
    for i in all_models:
        model_round = 'models/' + str(round_num) + '/' + str(i) + '_round_' + str(round_num) + '.pk1'
        if i == 'log':
             log = joblib.load(model_round)
             loaded_models.append(log)
        if i == 'svc':
             svc= joblib.load(model_round)
             loaded_models.append(svc)
        elif i == 'gmm':
            gmm = joblib.load(model_round)
            loaded_models.append(gmm)
        elif i == 'knn':
            kmeans = joblib.load(model_round)
            loaded_models.append(kmeans)
        elif i == 'gnb':
            gnb = joblib.load(model_round)
            loaded_models.append(gnb)
        
        print("Success :: Loaded - " + str(i))
        
    return loaded_models

# raw_X, raw_y = split_target(raw_data)

# display(raw_data.head())

pd.set_option('display.max_columns', 500)

formatted_data = raw_data.drop('points', 1)

def set_group(goals):
    if goals >= 2:
        return 1
    elif goals < 2:
        return 0
    
formatted_data['converted_goals'] = formatted_data.apply(lambda row: set_group(row['goals']), axis=1)

all_data = formatted_data

formatted_data = model_libs._clone_and_drop(formatted_data, ignore_cols)

rf = model_libs.reformat_formation(formatted_data, formatted_data.groupby('current_formation').groups, formatted_data.groupby('opp_formation').groups)

for r in range(len(rf)):
    formatted_data = formatted_data.replace(rf[r], r)

partially_formatted_data = formatted_data
formatted_data = formatted_data.drop('goals', 1)

(f_y, f_X) = model_libs._extract_target(formatted_data, 'converted_goals')
    
#display(formatted_data.head())

### FORMATTING DATA AS NEEDED

In [10]:
standard_cols = ['current_formation', 'is_home', 'avg_points', 'avg_goals_for', 'avg_goals_against', 'margin', 'goal_diff',
                'win_percentage', 'sos', 'opp_formation', 'opp_is_home', 'opp_avg_points', 'opp_avg_goals', 'opp_margin', 'opp_goal_diff', 'opp_win_percentage',
                'opp_opp_record', 'goals']
# 17 in each
home_cols = ['current_team_home_possession', 'current_team_home_attacks', 'current_team_home_dangerous_attacks', 'current_team_home_yellow_card',
            'current_team_home_corner_kicks', 'current_team_home_shots_on_target', 'current_team_home_shots_total', 'current_team_home_ball_safe',
            'current_team_home_played', 'current_opp_away_possession', 'current_opp_away_attacks', 'current_opp_away_dangerous_attacks', 'current_opp_away_yellow_card',
            'current_opp_away_corner_kicks', 'current_opp_away_shots_on_target', 'current_opp_away_shots_total', 'current_opp_away_ball_safe',
            'current_opp_away_played']
away_cols = ['current_team_away_possession', 'current_team_away_attacks', 'current_team_away_dangerous_attacks', 'current_team_away_yellow_card', 
            'current_team_away_corner_kicks', 'current_team_away_shots_on_target', 'current_team_away_shots_total', 'current_team_away_ball_safe', 
            'current_team_away_played', 'current_opp_home_possession', 'current_opp_home_attacks', 'current_opp_home_dangerous_attacks', 'current_opp_home_yellow_card', 'current_opp_home_corner_kicks', 
             'current_opp_home_shots_on_target', 'current_opp_home_shots_total', 'current_opp_home_ball_safe', 'current_opp_home_played']

extended_features = ['e_f_dangerous_attacks', 'e_f_shots_total', 'e_f_shots_on_target', 'e_f_ball_safe', 'e_f_possession', 'e_f_attacks',
               'opp_e_f_dangerous_attacks', 'opp_e_f_shots_total', 'opp_e_f_shots_on_target', 'opp_e_f_ball_safe', 'opp_e_f_possession', 'opp_e_f_attacks',
               'prev_opp_e_f_dangerous_attacks', 'prev_opp_e_f_shots_total', 'prev_opp_e_f_shots_on_target', 'prev_opp_e_f_ball_safe', 'prev_opp_e_f_possession', 'prev_opp_e_f_attacks',
               'opp_opp_e_f_dangerous_attacks', 'opp_opp_e_f_shots_total', 'opp_opp_e_f_shots_on_target', 'opp_opp_e_f_ball_safe', 'opp_opp_e_f_possession', 'opp_opp_e_f_attacks']

optional_extended_features = ['prev_opp_e_f_dangerous_attacks', 'prev_opp_e_f_shots_total', 'prev_opp_e_f_shots_on_target', 'prev_opp_e_f_ball_safe', 'prev_opp_e_f_possession', 'prev_opp_e_f_attacks',
               'opp_opp_e_f_dangerous_attacks', 'opp_opp_e_f_shots_total', 'opp_opp_e_f_shots_on_target', 'opp_opp_e_f_ball_safe', 'opp_opp_e_f_possession', 'opp_opp_e_f_attacks']

played_features = ['current_team_home_played', 'current_team_away_played','current_opp_home_played', 'current_opp_away_played', 'opp_is_home']


### RUNNING CLASSIFICATION MODEL ON IF TEAMS SCORE 0-1 OR 1-2 ON GAMES

In [7]:
def run_features(data, drop_data, target, models):
    
    new_data = data.drop(drop_data, axis=1)
    
    display(new_data.head())
    
    (y, X) = model_libs._extract_target(new_data, target)
    
    models = train_models(round_number, X, y, models)
    
    return models


#### Running ALL Features 
#models_test_1 = run_features(formatted_data, [], 'converted_goals', all_models)

#### Running only STANDARD Features
#models_test_2 = run_features(formatted_data, home_cols + away_cols + extended_features + played_features, 'converted_goals', all_models)

#### Running STANDARD and EXTENDED Features
#models_test_3 = run_features(formatted_data, home_cols + away_cols + optional_extended_features + played_features, 'converted_goals', all_models)

# Creates new column list for merged columns
merged_cols = []

for cols in home_cols:
    merged_cols.append(model_libs.rename_column(cols))
    
# Creates a Home and Away Table with Standard and Home/Away Columns.  Will merge together further down
#ha_data = model_libs._clone_and_drop(formatted_data, ignore_cols)
#display(reg_data.head())
home_data = formatted_data.loc[formatted_data.loc[:, 'is_home'] == 1, standard_cols + home_cols + extended_features + ['converted_goals']]
#display(home_data.head())
away_data = formatted_data.loc[formatted_data.loc[:, 'is_home'] == 0, standard_cols + away_cols + extended_features + ['converted_goals']]

# Combine the Home Table with the Away Table
ha_data = home_data.append(away_data)

ha_data = ha_data.drop('goals', axis=1)

# Combine Home/Away Columns into one for Current Team and Opponent.  Cuts down on Features
# Adds from whatever column that doesn't have Nan
for c in range(len(merged_cols)):
    ha_data[merged_cols[c]] = ha_data.apply(lambda row: model_libs.pick_column(row[home_cols[c]], row[away_cols[c]]), axis=1 )

ha_data = ha_data.drop(home_cols + away_cols, axis=1)

#### Running STANDARD and HOME and AWAY Features
#models_test_4 = run_features(ha_data, extended_features + ['current_opp_played', 'current_team_played'], 'converted_goals', ['log'])


models_test_1 = run_features(formatted_data, [], 'converted_goals', ['log'])
(f_y, f_X) = model_libs._extract_target(formatted_data, 'converted_goals')
log = models_test_1[0]
print(log.predict_proba(f_X))

actual_y = pd.DataFrame(f_y.values, columns=['actual'])
predictions = pd.DataFrame(log.predict(f_X), columns=['predictions'])
preds = pd.concat([predictions, actual_y], axis=1)
preds['diff'] = preds.apply(lambda r: model_libs.predictions_diff(r['predictions'], r['actual']), axis=1)
accuracy = np.divide(preds['diff'].sum(),float(len(preds['diff'])))
display(preds.head())
print(accuracy)


,is_home,current_formation,avg_points,avg_goals_for,avg_goals_against,margin,goal_diff,win_percentage,sos,opp_is_home,opp_formation,opp_avg_points,opp_avg_goals,opp_margin,opp_goal_diff,opp_win_percentage,opp_opp_record,current_team_home_possession,current_team_away_possession,current_team_home_attacks,current_team_away_attacks,current_team_home_dangerous_attacks,current_team_away_dangerous_attacks,current_team_home_yellow_card,current_team_away_yellow_card,current_team_home_corner_kicks,current_team_away_corner_kicks,current_team_home_shots_on_target,current_team_away_shots_on_target,current_team_home_shots_total,current_team_away_shots_total,current_team_home_ball_safe,current_team_away_ball_safe,current_team_home_played,current_team_away_played,current_opp_home_possession,current_opp_away_possession,current_opp_home_attacks,current_opp_away_attacks,current_opp_home_dangerous_attacks,current_opp_away_dangerous_attacks,current_opp_home_yellow_card,current_opp_away_yellow_card,current_opp_home_corner_kicks,current_opp_away_corner_kicks,current_opp_home_shots_on_target,current_opp_away_shots_on_target,current_opp_home_shots_total,current_opp_away_shots_total,current_opp_home_ball_safe,current_opp_away_ball_safe,current_opp_home_played,current_opp_away_played,e_f_dangerous_attacks,e_f_shots_total,e_f_shots_on_target,e_f_ball_safe,e_f_possession,e_f_attacks,opp_e_f_dangerous_attacks,opp_e_f_shots_total,opp_e_f_shots_on_target,opp_e_f_ball_safe,opp_e_f_possession,opp_e_f_attacks,prev_opp_e_f_dangerous_attacks,prev_opp_e_f_shots_total,prev_opp_e_f_shots_on_target,prev_opp_e_f_ball_safe,prev_opp_e_f_possession,prev_opp_e_f_attacks,opp_opp_e_f_dangerous_attacks,opp_opp_e_f_shots_total,opp_opp_e_f_shots_on_target,opp_opp_e_f_ball_safe,opp_opp_e_f_possession,opp_opp_e_f_attacks,converted_goals
0,0,2,3.00,2.000000,0.000000,2.000000,2,1.000000,1.166667,1,2,1.000000,3.000000,0.000000,0,0.500000,0.5000,46.0,48.195318,93.000000,96.025639,59.000000,46.561958,1.000000,1.916176,8.000000,4.184249,9.000000,4.131807,12.0,8.097396,87.0,92.56133,1,0,48.000000,48.195318,65,96.025639,44,46.561958,1,1.916176,5,4.184249,5.000000,4.131807,14,8.097396,66,92.56133,1,0,59.000000,12.000000,9.000000,87.000000,46.000000,93.000000,44.000000,14.000000,5.000000,66.000000,48.00,65.000000,55.000000,5.000000,1.000000,91.000000,54.000000,92.000000,51.000000,13.000000,6.000000,75.000000,52.000000,74.000000,0
1,1,2,1.50,1.000000,2.500000,-1.500000,-3,0.500000,2.000000,0,2,3.000000,3.000000,2.000000,4,1.000000,0.0000,46.0,52.000000,93.000000,97.000000,59.000000,51.000000,1.000000,3.000000,8.000000,7.000000,9.000000,4.000000,12.0,6.000000,87.0,71.00000,1,1,56.000000,43.000000,119,101.000000,45,49.000000,1,1.000000,3,2.000000,7.000000,6.000000,10,11.000000,119,102.00000,1,1,55.000000,9.000000,6.500000,79.000000,49.000000,95.000000,47.000000,10.500000,6.500000,110.500000,49.50,110.000000,44.000000,10.000000,5.000000,79.000000,48.000000,78.000000,68.500000,6.500000,1.500000,108.000000,46.000000,132.000000,1
2,0,2,2.00,1.333333,1.666667,-0.333333,-1,0.666667,0.880952,1,12,0.666667,0.666667,-1.000000,-3,0.357143,0.5000,49.5,52.000000,106.500000,97.000000,65.500000,51.000000,1.500000,3.000000,6.500000,7.000000,8.000000,4.000000,14.5,6.000000,92.5,71.00000,2,1,48.000000,45.000000,135,79.000000,73,38.500000,1,3.500000,7,4.500000,5.000000,3.000000,11,5.500000,121,81.00000,1,2,60.666667,11.666667,6.666667,85.333333,50.333333,103.333333,50.000000,7.333333,3.666667,94.333333,46.00,97.666667,45.333333,8.666667,5.333333,112.333333,48.666667,113.666667,52.666667,9.333333,5.333333,96.666667,52.333333,100.333333,1
3,1,12,2.25,1.750000,1.250000,0.500000,2,0.777778,0.500000,0,2,0.333333,0.666667,-0.666667,-2,0.166667,0.3125,49.5,49.500000,106.500000,108.000000,65.500000,48.500000,1.500000,2.500000,6.500000,4.500000,8.000000,5.000000,14.5,7.500000,92.5,93.00000,2,2,59.000000,56.000000,101,94.000000,60,67.000000,0,1.500000,8,7.000000,7.000000,5.500000,15,11.000000,98,75.00000,1,2,57.000000,

Training LOG REG Model
Score on Training Set :: 0.702702702703
Score on Test Set :: 0.568345323741
Finished LOG REG Modeling
[[ 0.81491713  0.18508287]
 [ 0.32479402  0.67520598]
 [ 0.09179552  0.90820448]
 ..., 
 [ 0.96125226  0.03874774]
 [ 0.99716678  0.00283322]
 [ 0.82196722  0.17803278]]


,predictions,actual,diff
0,0,0,1
1,1,1,1
2,1,1,1
3,0,0,1
4,1,1,1


0.675792507205


# Optimize Features

Since STANDARD and HOME and AWAY Features got the best score going to optimize and minimize off of those features.

In [ ]:
def plot_RFECV(X, y):
    svc = SVC(kernel="linear")
    rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(y, 2),
                  scoring='accuracy')
    rfecv.fit(X, y)

    # Plot number of features VS. cross-validation scores
    plt.figure()
    plt.xlabel("Number of features selected")
    plt.ylabel("Cross validation score (nb of correct classifications)")
    plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
    plt.show()
    
plot_RFECV(ha_X, ha_y)

In [ ]:
def create_RFE(X, y):
    # Create the RFE object and rank each pixel
    svc = SVC(kernel="linear", C=1)
    rfe = RFE(estimator=svc, n_features_to_select=1, step=1)
    rfe.fit(X, y)
    print(rfe.ranking_)
    return rfe.ranking_

(ha_y, ha_X) = model_libs._extract_target(ha_data, 'converted_goals')

# Taking all of Home/Away Data (even the Extended Features)
rankings = create_RFE(ha_X, ha_y)

In [ ]:
""" Reordering the columns in the data to reflect the rankings """

rankings_name = []
ranked_X = ha_X

for r in range(len(rankings)):
    column_name = ha_X.iloc[:, rankings[r]-1].name
    rankings_name.append(column_name)

ranked_X = ranked_X.reindex_axis(rankings_name, axis=1)
display(ranked_X.head())

In [ ]:
def train_ranked_models(X, y):
    
    for x in xrange(10, 20):
        print('X :: {}'.format(x))
        ranked_models = train_models(round_number, X.iloc[:, 1:x], y, ['knn'])
    
train_ranked_models(ranked_X, ha_y)

In [ ]:
def plot_PCA(data, preds):
    
    pca = PCA(n_components=4)
    pca.fit(data)
    
    # Generate PCA results plot
    pca_results = rs.pca_results(data, pca)
    #print(pca_results)
    pca_data = pca.transform(data)
    
    # Create a DataFrame for the reduced data
    pca_data = pd.DataFrame(pca_data, columns = ['Dimension 1', 'Dimension 2', 'Dimension 3', 'Dimension 4'])
    plot_data = pd.concat([preds, pca_data], axis = 1)
    
    # Scatterplotting the transformed data if it's 2 Dimensions
    """fig, ax = plt.subplots(figsize = (14,8))
    cmap = cm.get_cmap('gist_rainbow')
    
    for i, cluster in plot_data.groupby('converted_goals'):   
        cluster.plot(ax = ax, kind = 'scatter', x = 'Dimension 1', y = 'Dimension 2', 
                     color = cmap((i)*1.0/(4)), label = 'Points %i'%(i), s=30);"""
    
    return pca, plot_data, pca_results

#(ha_y, ha_X) = model_libs._extract_target(ha_data, 'converted_goals') 

pca, pca_data, pca_results = plot_PCA(ha_X, ha_y)
#display(pca_data.head())

print(pca_data.shape)
# Runs models on PCA to see if there is a difference in the results
models = run_features(pca_data, [], 'converted_goals', ['knn'])
(pca_y, pca_X) = model_libs._extract_target(pca_data, 'converted_goals')
knn = models[0]
#preds = knn.predict(pca_X)

actual_y = pd.DataFrame(pca_y.values, columns=['actual'])
predictions = pd.DataFrame(knn.predict(pca_X), columns=['predictions'])
preds = pd.concat([predictions, actual_y], axis=1)
preds['diff'] = preds.apply(lambda r: model_libs.predictions_diff(r['predictions'], r['actual']), axis=1)
accuracy = np.divide(preds['diff'].sum(),float(len(preds['diff'])))
print(accuracy)


In [ ]:
display(pca_results)
print(type(pca_results))
print(pca_results[])

### Making Predictions on Expected Goals Final Selected Model

In [8]:
(ha_y, ha_X) = model_libs._extract_target(ha_data, 'converted_goals')
knn = train_models(round_number, ha_X, ha_y, ['knn'])
actual_y = pd.DataFrame(ha_y.values, columns=['actual'])

predictions = pd.DataFrame(knn[0].predict(ha_X), columns=['expected_goal_category'])
preds = pd.concat([predictions, actual_y], axis=1)
preds['diff'] = preds.apply(lambda r: model_libs.predictions_diff(r['expected_goal_category'], r['actual']), axis=1)
accuracy = np.divide(preds['diff'].sum(),float(len(preds['diff'])))
print(accuracy)

-----------------------------------
Training K Neighbors Classifier Model
KNN Score :: 0.755043227666
Finished K-Means Modeling
0.755043227666


======================================================================================================================

### RUNNING LINEAR REGRESSION on FEATURES

### Creating Regression Model independent from Classification (without the Classifier's input as a feature)

In [16]:
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split, cross_val_predict
from sklearn import linear_model
from sklearn import grid_search
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression
from sklearn.cross_validation import cross_val_score

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)

# Can't use formatted_data.  Still need the goals column
regr_data = all_data

""" Need to add expected and opp expected goals (categorically set) """
# Need to add results of the classifier to the features
regr_data['expected_goals'] = predictions
regr_data['opp_expected_goals'] = pd.Series('', index=regr_data.index)

holder_data = regr_data.copy()

""" Using a copy of regr_data, iterating through the df and finding the 
   reverse match (same match but with the teams opposite) """
for i, match in holder_data.iterrows():
    if holder_data.loc[i, 'opp_expected_goals'] == '':
        match_id = match.loc['match_id']
        curr_goals = match['expected_goals']
        reverse_match = holder_data[(holder_data['team_id'] == match.loc['opp_id']) & (holder_data['opp_id'] == match.loc['team_id'])]
        reverse_match = reverse_match[reverse_match['match_id'] == match_id]
        regr_data.loc[reverse_match.index.values[0], 'opp_expected_goals'] = curr_goals
    
""" Need to make Home/Away Columns on new DF """
# Creates new column list for merged columns
merged_cols = []

for cols in home_cols:
    merged_cols.append(model_libs.rename_column(cols))
    
# Creates a Home and Away Table with Standard and Home/Away Columns.  Will merge together further down
home_data = regr_data.loc[regr_data.loc[:, 'is_home'] == 1, standard_cols + home_cols + extended_features + ['converted_goals', 'expected_goals', 'opp_expected_goals']]
away_data = regr_data.loc[regr_data.loc[:, 'is_home'] == 0, standard_cols + away_cols + extended_features + ['converted_goals', 'expected_goals', 'opp_expected_goals']]

# Combine the Home Table with the Away Table
regr_data = home_data.append(away_data)

# Combine Home/Away Columns into one for Current Team and Opponent.  Cuts down on Features
# Adds from whatever column that doesn't have Nan
for c in range(len(merged_cols)):
    regr_data[merged_cols[c]] = regr_data.apply(lambda row: model_libs.pick_column(row[home_cols[c]], row[away_cols[c]]), axis=1 )
    
# Removes the unneeded Home/Away Columns
regr_data = regr_data.drop(home_cols + away_cols, axis=1)

# Both for current team and the current opponent
regr_data = model_libs._clone_and_drop(regr_data, ignore_cols + ['points'])

rf = model_libs.reformat_formation(regr_data, regr_data.groupby('current_formation').groups, regr_data.groupby('opp_formation').groups)

for r in range(len(rf)):
    regr_data = regr_data.replace(rf[r], r)

    
# What other data can we remove that's not useful to predicting the number of goals per game
regr_data = regr_data.drop(optional_extended_features + ['converted_goals', 'goal_diff', 'opp_avg_goals', 'opp_avg_points', 
                                                         'current_team_yellow_card', 'current_team_corner_kicks',
                                                        'opp_goal_diff', 'opp_margin', 'opp_opp_record'
                                                        ,'current_opp_played', 'current_team_played', 'current_opp_yellow_card', 'current_opp_corner_kicks', 
                                                         'current_opp_played', 'current_opp_attacks', 'current_opp_dangerous_attacks', 'current_opp_shots_on_target'
                                                        , 'current_opp_shots_total'], axis=1)

display(regr_data.head())

(regr_y, regr_X) = model_libs._extract_target(regr_data, 'goals')
        
X_train, X_test, y_train, y_test = cross_validation.train_test_split(regr_X, regr_y, test_size=0.2, random_state=42)

lin_regr = linear_model.LinearRegression()
lin_regr.fit(X_train, y_train)
#predicted = cross_val_predict(lin_regr, reg_X, reg_y, cv=10)
print('Coefficients: \n', lin_regr.coef_)
print("Residual sum of squares: %.2f"
      % np.mean((lin_regr.predict(X_test) - y_test) ** 2))

preds = pd.DataFrame(lin_regr.predict(X_test), columns=['predictions'])
preds['rounded'] = preds.loc[0:].apply(lambda r: np.round(r), axis=1)
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % lin_regr.score(X_test, preds['rounded']))
display(preds.head())
preds = pd.concat([preds, pd.DataFrame(y_test.as_matrix(), columns=['actual'])], axis = 1)
preds['correct'] = preds.apply(lambda r: model_libs.predictions_diff(r['rounded'], r['actual']), axis=1)
preds['correct_category'] = preds.apply(lambda r: model_libs.check_category(r['rounded'], r['actual']), axis=1)
#display(preds.head())
accuracy = np.divide(preds['correct'].sum(),float(len(preds['correct'])))
accuracy_category = np.divide(preds['correct_category'].sum(), float(len(preds['correct_category'])))
print('accuracy on X TEST')
print(accuracy)
print('accuracy_category on X TEST')
print(accuracy_category)


,avg_goals_against,avg_goals_for,avg_points,current_formation,e_f_attacks,e_f_ball_safe,e_f_dangerous_attacks,e_f_possession,e_f_shots_on_target,e_f_shots_total,expected_goals,goals,is_home,margin,opp_e_f_attacks,opp_e_f_ball_safe,opp_e_f_dangerous_attacks,opp_e_f_possession,opp_e_f_shots_on_target,opp_e_f_shots_total,opp_expected_goals,opp_formation,opp_is_home,opp_win_percentage,sos,win_percentage,current_team_possession,current_team_attacks,current_team_dangerous_attacks,current_team_shots_on_target,current_team_shots_total,current_team_ball_safe,current_opp_possession,current_opp_ball_safe
1,2.500000,1.000000,1.500000,2,95.000000,79.000000,55.000000,49.000000,6.500000,9.000000,0,2,1,-1.500000,110.000000,110.500000,47.000000,49.50,6.5,10.500000,0,2,0,1.000000,2.000000,0.500000,46.0,93.000000,59.000000,9.000000,12.00,87.0,43.0,102.0
3,1.250000,1.750000,2.250000,12,107.250000,92.750000,57.000000,49.500000,6.500000,11.000000,0,1,1,0.500000,96.333333,82.666667,64.666667,57.00,6.0,12.333333,0,2,0,0.166667,0.500000,0.777778,49.5,106.500000,65.500000,8.000000,14.50,92.5,56.0,75.0
4,1.200000,1.600000,2.000000,12,105.800000,88.800000,55.600000,49.200000,6.400000,12.600000,0,2,1,0.400000,99.250000,95.750000,48.000000,49.25,3.5,5.500000,0,12,0,0.617647,1.388803,0.717391,49.0,104.333333,60.333333,7.333333,16.00,86.0,45.0,89.0
6,1.333333,1.666667,1.833333,12,103.333333,87.666667,54.500000,50.833333,6.166667,12.666667,0,2,1,0.333333,118.000000,104.400000,56.400000,51.20,3.6,8.000000,1,7,0,0.769231,1.678673,0.678571,51.5,101.000000,57.500000,6.750000,15.25,85.0,45.5,109.0
10,1.636364,1.363636,1.545455,2,96.818182,89.272727,53.272727,51.272727,5.363636,11.181818,0,2,1,-0.272727,97.500000,93.500000,50.100000,46.30,4.9,8.700000,0,7,0,0.433333,1.047534,0.580000,51.0,100.400000,55.600000,6.400000,14.00,88.6,46.8,101.2


('Coefficients: \n', array([ 0.00215839, -0.00986285, -0.3592211 , -0.02519152,  0.00208923,
        0.00709401, -0.00902571,  0.01956322, -0.02088338,  0.01043588,
        0.0404976 ,  0.22643997, -0.01202124, -0.02464894,  0.00855452,
        0.02658425, -0.00340269, -0.08049008, -0.03641006, -0.17125964,
       -0.00368989, -0.22643997,  0.77711045, -0.31221408,  1.1347224 ,
        0.00345844,  0.00835685,  0.00455593, -0.04309334,  0.00706675,
       -0.01171106,  0.01267741,  0.00213764]))
Residual sum of squares: 1.39
Variance score: 0.67


,predictions,rounded
0,0.860042,1
1,1.615077,2
2,1.658405,2
3,1.719000,2
4,1.944588,2


accuracy on X TEST
0.345323741007
accuracy_category on X TEST
0.553956834532


Predicting all goals on the regression model

In [18]:
#k_X = pd.DataFrame(k_X)

all_preds = pd.DataFrame(lin_regr.predict(regr_X), columns=['predictions'])
all_preds['rounded'] = all_preds.loc[0:].apply(lambda r: np.round(r), axis=1)

all_preds = pd.concat([regr_X, all_preds], axis = 1)

all_preds = pd.concat([all_preds, regr_data['goals']], axis=1)

necessary = all_preds[['predictions', 'rounded', 'goals']]
    
necessary['correct'] = necessary.apply(lambda r: model_libs.predictions_diff(r['rounded'], r['goals']), axis=1)
necessary['correct_category'] = necessary.apply(lambda r: model_libs.check_category(r['predictions'], r['goals']), axis=1)
# display(necessary)

accuracy = np.divide(necessary['correct'].sum(),float(len(necessary['correct'])))
accuracy_category = np.divide(necessary['correct_category'].sum(), float(len(necessary['correct_category'])))

print('accuracy on All of X')
print(accuracy)
print('accuracy_category on All of X')
print(accuracy_category)

accuracy on All of X
0.337175792507
accuracy_category on All of X
0.596541786744


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
